In [2]:
import pandas as pd
import numpy as np

In [22]:
#train=pd.read_csv('train_full.csv',index_col=[0])
train=pd.read_csv('train.csv')

In [23]:
train.shape

(1460, 81)

In [24]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [25]:
pd.set_option('display.max_rows', None)
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [6]:
def root_mean_squared_log_error(y_valid, y_preds):
    """Calculate root mean squared error of log(y_true) and log(y_pred)"""
    if len(y_preds)!=len(y_valid): return 'error_mismatch'
    y_preds_new = [math.log(x) for x in y_preds]
    y_valid_new = [math.log(x) for x in y_valid]
    return mean_squared_error(y_valid_new, y_preds_new, squared=False)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [26]:
x=pd.DataFrame(train.LotArea)
x.rename(columns={'LotArea':'sqrt'},inplace=True)
x['bath']=train.FullBath
x['cond']=train.OverallCond
x['location']=train.Neighborhood
x

,sqrt,bath,cond,location
0,8450,2,5,CollgCr
1,9600,2,8,Veenker
2,11250,2,5,CollgCr
3,9550,1,5,Crawfor
4,14260,2,5,NoRidge
5,14115,1,5,Mitchel
6,10084,2,5,Somerst
7,10382,2,6,NWAmes
8,6120,2,5,OldTown
9,7420,1,6,BrkSide


In [ ]:
dic={'Blmngtn':'Bloomington Heights',
       'Blueste':'Bluestem',
       'BrDale':'Briardale',
       'BrkSide':'Brookside',
       'ClearCr':'Clear Creek',
       'CollgCr':'College Creek',
       'Crawfor':'Crawford',
       'Edwards':'Edwards',
       'Gilbert':'Gilbert',
       'IDOTRR':'Iowa Rail Road',
       'MeadowV':'Meadow Village',
       'Mitchel':'Mitchell',
       'Names':'North Ames',
       'NoRidge':'Northridge',
       'NPkVill':'Northpark Villa',
       'NridgHt':'Northridge Heights',
       'NWAmes':'Northwest Ames',
       'OldTown':'Old Town',
       'SWISU':'Iowa State University',
       'Sawyer':'Sawyer',
       'SawyerW':'Sawyer West',
       'Somerst':'Somerset',
       'StoneBr':'Stone Brook',
       'Timber':'Timberland',
       'Veenker':'Veenker'
    }

In [ ]:
x.replace({'location':dic})

In [ ]:
x.head()

In [29]:
x.location.value_counts()

NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
BrkSide     58
Crawfor     51
Mitchel     49
NoRidge     41
Timber      38
IDOTRR      37
ClearCr     28
SWISU       25
StoneBr     25
Blmngtn     17
MeadowV     17
BrDale      16
Veenker     11
NPkVill      9
Blueste      2
Name: location, dtype: int64

In [27]:
y=pd.DataFrame(train.SalePrice)
y.rename(columns={'SalePrice':'price'},inplace=True)
y

,price
0,208500
1,181500
2,223500
3,140000
4,250000
5,143000
6,307000
7,200000
8,129900
9,118000


In [41]:
y.price.dtype

dtype('int64')

In [28]:
dummies=pd.get_dummies(x.location)
dummies.head()

,Blmngtn,Blueste,BrDale,BrkSide,ClearCr,CollgCr,Crawfor,Edwards,Gilbert,IDOTRR,...,NoRidge,NridgHt,OldTown,SWISU,Sawyer,SawyerW,Somerst,StoneBr,Timber,Veenker
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [32]:
xx=pd.concat([x,dummies.drop('Blueste',axis='columns')],axis='columns')
xx.head()

,sqrt,bath,cond,location,Blmngtn,BrDale,BrkSide,ClearCr,CollgCr,Crawfor,...,NoRidge,NridgHt,OldTown,SWISU,Sawyer,SawyerW,Somerst,StoneBr,Timber,Veenker
0,8450,2,5,CollgCr,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,9600,2,8,Veenker,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,11250,2,5,CollgCr,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,9550,1,5,Crawfor,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,14260,2,5,NoRidge,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [34]:
X=xx.drop('location',axis='columns')
X.head()

,sqrt,bath,cond,Blmngtn,BrDale,BrkSide,ClearCr,CollgCr,Crawfor,Edwards,...,NoRidge,NridgHt,OldTown,SWISU,Sawyer,SawyerW,Somerst,StoneBr,Timber,Veenker
0,8450,2,5,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9600,2,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,11250,2,5,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9550,1,5,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,14260,2,5,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [37]:
from sklearn.linear_model import LinearRegression
X_train, X_valid, y_train, y_valid=train_test_split(X,y,test_size=0.2,random_state=73)
lr_clf=LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_valid,y_valid)

0.6567642186529392

In [44]:
def predict_price(location,sqft,bath,cond):
    loc_index=np.where(X.columns==location)[0][0]
    
    x=np.zeros(len(X.columns))
    x[0]=sqft
    x[1]=bath
    x[2]=cond
    if loc_index>=0:
        x[loc_index]=1
        
    return lr_clf.predict([x])[0]

In [45]:
predict_price('NAmes',1000,2,5)

array([158742.96077702])

In [46]:
import pickle
with open('home_prices_model.pickle','wb')as f:
    pickle.dump(lr_clf,f)

In [47]:
import json
columns={'data_columns':[col.lower() for col in X.columns]}
with open('columns.json','w') as f:
    f.write(json.dumps(columns))

In [7]:
y=train['SalePrice']
x=train.drop(['SalePrice'],axis=1)

In [8]:
X_train, X_valid, y_train, y_valid=train_test_split(x,y,random_state=73)

In [9]:
dtr=DecisionTreeRegressor(random_state=73)

In [10]:
dtr.fit(X_train,y_train)

DecisionTreeRegressor(random_state=73)

In [11]:
y_pred=dtr.predict(X_valid)

In [12]:
print('RMSLE:', root_mean_squared_log_error(y_valid, y_pred))

RMSLE: 0.20458343032399479


In [13]:
RF=RandomForestRegressor(n_estimators=1000,random_state=73)
RF.fit(X_train,y_train)
y_pred_RF=RF.predict(X_valid)
print('RMSLE:', root_mean_squared_log_error(y_valid, y_pred_RF))

RMSLE: 0.144563283965543


In [14]:
XGB=XGBRegressor(n_estimators=1200,learning_rate=0.05,random_state=73)
XGB.fit(X_train,y_train)
y_pred_XGB=XGB.predict(X_valid)
print('RMSLE:', root_mean_squared_log_error(y_valid, y_pred_XGB))

RMSLE: 0.12949802518955733


In [16]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

grid = GridSearchCV(XGB, param_grid=params, n_jobs=4, cv=5, verbose=3 )
grid.fit(X_train,y_train)
print('\n All results:')
print(grid.cv_results_)
print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best score:')
print(grid.best_score_ * 2 - 1)
print('\n Best parameters:')
print(grid.best_params_)

Fitting 5 folds for each of 405 candidates, totalling 2025 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   44.3s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  3.5min
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed: 10.0min
[Parallel(n_jobs=4)]: Done 504 tasks      | elapsed: 20.7min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 35.7min
[Parallel(n_jobs=4)]: Done 1144 tasks      | elapsed: 56.4min
[Parallel(n_jobs=4)]: Done 1560 tasks      | elapsed: 81.1min
[Parallel(n_jobs=4)]: Done 2025 out of 2025 | elapsed: 111.3min finished



 All results:
{'mean_fit_time': array([ 5.28791022,  5.22774029,  5.03743391,  5.82473121,  6.46083055,
        6.0941524 ,  6.3044138 ,  5.52693949,  5.00374436,  7.01821661,
        6.68041568,  6.2894733 ,  6.77185974,  6.56009483,  6.48429289,
        6.93238759,  6.71570983,  6.26520853,  8.30183997,  8.29762912,
        8.18442764,  8.53796973,  8.14732842,  7.59776254,  9.58556337,
        8.5661447 ,  7.44786286,  5.48472753,  5.30895848,  5.03903837,
        5.4262712 ,  5.27968006,  5.03432641,  5.56905169,  5.33091707,
        5.03984041,  8.60502234, 11.39066072, 10.76472454, 11.46227584,
       11.20226574, 10.55610194, 11.22280302, 10.99872808, 10.6751039 ,
       14.04891992, 13.61184964, 13.00402646, 13.74817038, 13.20211582,
       12.85438552, 13.6944242 , 13.2457654 , 12.92341866, 10.05905676,
        8.94239144,  8.53375797,  9.0869297 ,  8.94904532,  8.48223853,
        9.01601706,  8.89788098,  8.4590847 , 11.29818144, 11.30655251,
       11.40737062, 11.42050457

             tree_method='exact', validate_parameters=1, verbosity=None)

 Best score:
0.7504308311196255

 Best parameters:
{'colsample_bytree': 0.6, 'gamma': 0.5, 'max_depth': 4, 'min_child_weight': 1, 'subsample': 1.0}


In [21]:
hp_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0.5, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=4,
             min_child_weight=1, monotone_constraints='()',
             n_estimators=1200, n_jobs=0, num_parallel_tree=1, random_state=73,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1.0,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [22]:
hp_model.fit(X_train,y_train)
y_pred_hp_model=hp_model.predict(X_valid)
print('RMSLE:', root_mean_squared_log_error(y_valid, y_pred_hp_model))

RMSLE: 0.1206568336832177


In [24]:
params = {
        'n_estimators': [900,1000,1100,1200],
        'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06],
        'min_child_weight': [0.5, 1, 2],
        'gamma': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7],
        'subsample': [0.9, 1.0, 1.1,1.2],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8],
        'max_depth': [3, 4, 5]
        }

grid = GridSearchCV(XGB, param_grid=params, n_jobs=4, cv=5, verbose=3 )
grid.fit(X_train,y_train)
#print('\n All results:')
#print(grid.cv_results_)
print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best score:')
print(grid.best_score_ * 2 - 1)
print('\n Best parameters:')
print(grid.best_params_)

Fitting 5 folds for each of 25920 candidates, totalling 129600 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   40.7s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  1.9min
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:  4.3min
[Parallel(n_jobs=4)]: Done 504 tasks      | elapsed:  8.1min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 13.4min
[Parallel(n_jobs=4)]: Done 1144 tasks      | elapsed: 18.7min
[Parallel(n_jobs=4)]: Done 1560 tasks      | elapsed: 26.1min
[Parallel(n_jobs=4)]: Done 2040 tasks      | elapsed: 34.3min
[Parallel(n_jobs=4)]: Done 2584 tasks      | elapsed: 43.2min
[Parallel(n_jobs=4)]: Done 3192 tasks      | elapsed: 53.5min
[Parallel(n_jobs=4)]: Done 3864 tasks      | elapsed: 83.6min
[Parallel(n_jobs=4)]: Done 4600 tasks      | elapsed: 90.7min
[Parallel(n_jobs=4)]: Done 5400 tasks      | elapsed: 98.0min
[Parallel(n_jobs=4)]: Done 6264 tasks      | elapsed: 106.0min
[Parallel(n_jobs=4)]: Done 7192 tasks      | elapsed: 115.0mi


 Best estimator:
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0.2, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=4,
             min_child_weight=2, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=73,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1.0,
             tree_method='exact', validate_parameters=1, verbosity=None)

 Best score:
0.7564851811774047

 Best parameters:
{'colsample_bytree': 0.5, 'gamma': 0.2, 'learning_rate': 0.05, 'max_depth': 4, 'min_child_weight': 2, 'n_estimators': 1000, 'subsample': 1.0}


In [28]:
hp_model_f=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0.2, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=4,
             min_child_weight=2, monotone_constraints='()',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=73,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1.0,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [29]:
hp_model_f.fit(X_train,y_train)
y_pred_hp_model_f=hp_model.predict(X_valid)
print('RMSLE:', root_mean_squared_log_error(y_valid, y_pred_hp_model_f))

RMSLE: 0.1206568336832177


In [20]:
model_xgb.fit(X_train,y_train)
y_pred_model_xgb=model_xgb.predict(X_valid)
print('RMSLE:', root_mean_squared_log_error(y_valid, y_pred_model_xgb))

[00:20:25] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


RMSLE: 0.13199381560457535


In [ ]:
df = pd.DataFrame(columns=('w1', 'w2', 'w3', 'mean', 'std'))

i = 0
for w1 in range(1,4):
    for w2 in range(1,4):
        for w3 in range(1,4):

            if len(set((w1,w2,w3))) == 1: # skip if all weights are equal
                continue

            eclf = EnsembleClassifier(clfs=[clf1, clf2, clf3], weights=[w1,w2,w3])
            scores = cross_validation.cross_val_score(
                                            estimator=eclf,
                                            X=X,
                                            y=y,
                                            cv=5,
                                            scoring='accuracy',
                                            n_jobs=1)

            df.loc[i] = [w1, w2, w3, scores.mean(), scores.std()]
            i += 1

df.sort(columns=['mean', 'std'], ascending=False)